veri görselleştirme https://batuhandaz.medium.com/decision-tree-algoritmas%C4%B1-karar-a%C4%9Fac%C4%B1-machine-learning-78d856b1f457
https://ece-akdagli.medium.com/makine-%C3%B6%C4%9Frenmesinde-decision-tree-42a86502ee75
https://erdincuzun.com/makine_ogrenmesi/decision-tree-karar-agaci-id3-algoritmasi-classification-siniflama/

### Import Librarires And Dataset

In [33]:
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#some settings to show data
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# target_url = ("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data")
target_url = ("datasets/abalone.data")
abalone_df = pd.read_csv(target_url)
headers = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']
abalone_df.columns = headers

### Analyze the Data

In [34]:
abalone_df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8


In [35]:
abalone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4176 entries, 0 to 4175
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4176 non-null   object 
 1   Length          4176 non-null   float64
 2   Diameter        4176 non-null   float64
 3   Height          4176 non-null   float64
 4   Whole weight    4176 non-null   float64
 5   Shucked weight  4176 non-null   float64
 6   Viscera weight  4176 non-null   float64
 7   Shell weight    4176 non-null   float64
 8   Rings           4176 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


**Get target value**

In [36]:
# If you want the target values to be categorical rather than numeric, this process should be applied.

# for ix in abalone_df.index:
#     row = abalone_df.loc[ix]
#     if row["Rings"] <= 8:
#         abalone_df.loc[ix, 'Rings'] = 'Young'
#     elif row["Rings"] >= 11:
#         abalone_df.loc[ix, 'Rings'] = 'Old'
#     elif row["Rings"] >=9 & row["Rings"] <= 10:
#         abalone_df.loc[ix, 'Rings'] = 'Medium'
        
# X = abalone.drop(columns="Rings")
# y = abalone["Rings"]
# X['Sex'] = abalone['Sex'].replace({'F':0,'M':1,'I':2})

In [37]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [38]:
def is_numeric_value(x):
    return type(x) == int or type(x) == float

In [39]:
abalone_list = abalone_df.values.tolist()
split_index = int(0.3 * len(abalone_list))

test = abalone_list[:split_index:]
train = abalone_list[split_index:]

In [40]:
class MyQuestioner:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def __repr__(self):
        status = "=="
        if is_numeric_value(self.value):
            status = ">="
        return f"Is {headers[self.column]} {status} {self.value}"

    def compare(self, compared):
        val = compared[self.column]
        if is_numeric_value(val):
            return val >= self.value
        else:
            return val == self.value

In [41]:
print(MyQuestioner(0, "F"))
print(MyQuestioner(0, 19))

print(MyQuestioner(2, 19))
print(MyQuestioner(2, "19"))

Is Sex == F
Is Sex >= 19
Is Diameter >= 19
Is Diameter == 19


In [42]:
compared = train[0]
q = MyQuestioner(0, "M")
print(q.compare(compared))

q = MyQuestioner(0, 12)
print(q.compare(compared))

False
False


In [43]:
def partitioner(rows, myQuestion):
    true_rows, false_rows = [], []
    for row in rows:
        if myQuestion.compare(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [44]:
true_rows, false_rows = partitioner(train, MyQuestioner(0, 'M'))
true_rows

[['M', 0.495, 0.395, 0.12, 0.553, 0.224, 0.1375, 0.167, 8],
 ['M', 0.5, 0.42, 0.135, 0.6765, 0.302, 0.1415, 0.2065, 9],
 ['M', 0.525, 0.42, 0.155, 0.842, 0.428, 0.1415, 0.2045, 9],
 ['M', 0.535, 0.435, 0.15, 0.717, 0.3475, 0.1445, 0.194, 9],
 ['M', 0.55, 0.43, 0.15, 0.8745, 0.413, 0.1905, 0.248, 9],
 ['M', 0.555, 0.44, 0.14, 0.8705, 0.407, 0.156, 0.255, 9],
 ['M', 0.565, 0.445, 0.15, 0.796, 0.3635, 0.184, 0.219, 8],
 ['M', 0.565, 0.39, 0.125, 0.744, 0.352, 0.13, 0.1685, 11],
 ['M', 0.57, 0.435, 0.17, 0.873, 0.382, 0.183, 0.2705, 10],
 ['M', 0.57, 0.435, 0.125, 0.8965, 0.383, 0.1835, 0.275, 9],
 ['M', 0.575, 0.46, 0.155, 0.892, 0.4415, 0.176, 0.22, 10],
 ['M', 0.58, 0.46, 0.155, 1.4395, 0.6715, 0.273, 0.2955, 10],
 ['M', 0.58, 0.455, 0.135, 0.7955, 0.405, 0.167, 0.204, 10],
 ['M', 0.585, 0.465, 0.155, 0.9145, 0.4555, 0.1965, 0.235, 9],
 ['M', 0.585, 0.49, 0.185, 1.171, 0.522, 0.2535, 0.335, 10],
 ['M', 0.585, 0.46, 0.165, 1.1135, 0.5825, 0.2345, 0.274, 10],
 ['M', 0.585, 0.47, 0.165, 1.

In [45]:
false_rows

[['I', 0.42, 0.305, 0.09, 0.328, 0.168, 0.0615, 0.082, 6],
 ['I', 0.425, 0.325, 0.11, 0.3335, 0.173, 0.045, 0.1, 7],
 ['I', 0.425, 0.32, 0.1, 0.3055, 0.126, 0.06, 0.106, 7],
 ['I', 0.425, 0.31, 0.09, 0.301, 0.1385, 0.065, 0.08, 7],
 ['I', 0.43, 0.34, 0.0, 0.428, 0.2065, 0.086, 0.115, 8],
 ['I', 0.43, 0.315, 0.095, 0.378, 0.175, 0.08, 0.1045, 8],
 ['I', 0.435, 0.315, 0.11, 0.3685, 0.1615, 0.0715, 0.12, 7],
 ['I', 0.44, 0.34, 0.12, 0.438, 0.2115, 0.083, 0.12, 9],
 ['I', 0.45, 0.33, 0.105, 0.448, 0.208, 0.089, 0.12, 9],
 ['I', 0.455, 0.345, 0.105, 0.4005, 0.164, 0.0755, 0.126, 8],
 ['F', 0.455, 0.365, 0.115, 0.4305, 0.184, 0.108, 0.1245, 8],
 ['I', 0.455, 0.33, 0.1, 0.372, 0.358, 0.0775, 0.11, 8],
 ['I', 0.46, 0.36, 0.105, 0.466, 0.2225, 0.099, 0.11, 7],
 ['I', 0.46, 0.35, 0.105, 0.3705, 0.1575, 0.077, 0.114, 9],
 ['F', 0.46, 0.365, 0.125, 0.4785, 0.206, 0.1045, 0.141, 8],
 ['I', 0.465, 0.34, 0.11, 0.346, 0.1425, 0.073, 0.113, 11],
 ['I', 0.47, 0.365, 0.1, 0.411, 0.175, 0.0855, 0.135, 8],

In [46]:
def class_counts(rows):
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        label = row[-1] # in our dataset format, the label is always the last column
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [47]:
counts = class_counts(train)
print(counts)

{6: 168, 7: 261, 8: 437, 9: 529, 11: 371, 13: 132, 10: 482, 12: 179, 15: 51, 14: 76, 3: 7, 4: 32, 5: 68, 16: 38, 17: 30, 19: 15, 27: 2, 18: 24, 20: 10, 21: 5, 25: 1, 23: 4, 24: 2}


In [48]:
def gini_impurity(rows): # There are some ways like entropy, but I use gini impurity : 
    #  https://medium.com/machine-learning-t%C3%BCrkiye/karar-agaclari-algoritmasi-b823c23997d0#:~:text=Gini%20impurity%20nedir,%C3%B6l%C3%A7mek%20i%C3%A7in%20kullan%C4%B1labilir.
    
    impurityValue = 1
    counts = class_counts(rows)
    
    for label in counts:
        probability_of_label = counts[label] / float(len(rows))
        impurityValue -= probability_of_label ** 2
        
    return impurityValue

In [49]:
current_uncertainty = gini_impurity(train)
current_uncertainty

0.8825754031450648

In [50]:
def information_gain(left, right, current_uncertainty):
    # Information Gain: The uncertainty of the starting node, minus the weighted impurity of two child nodes.

    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - (1 - p) * gini_impurity(right) - p * gini_impurity(left)

In [51]:
# How much information do we gain by partioning on 'F (Female)' ?
true_rows, false_rows = partitioner(train, MyQuestioner(0, "F"))
information_gain(true_rows, false_rows, current_uncertainty)

0.006033667611677618

In [52]:
def find_the_best_split(rows):
    # Find the best question to ask by iterating over every feature / value and calculating the information gain.
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    n_features = len(rows[0]) - 1  # number of columns
    current_uncertainty = gini_impurity(rows)

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = MyQuestioner(col, val)          
            true_rows, false_rows = partitioner(rows, question)  # try splitting the dataset

            if len(true_rows) == 0 or len(false_rows) == 0: # Skip this split if it doesn't divide the dataset.
                continue

            gain = information_gain(true_rows, false_rows, current_uncertainty) # Calculate the information gain from this split

            # You actually can use '>=' instead of '>' here, but I wanted the tree to look a certain way for our dataset.
            if gain > best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [53]:
best_gain, best_question = find_the_best_split(train)
best_question

Is Shell weight >= 0.1775

In [54]:
class Leaf:
    # A Leaf node classifies data: so this holds a dictionary of class ( for us Old, Mediun or Young) 
    #   -> number of times it appears in the rows from the training data that reach this leaf.

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [55]:
class Decision_Node:
    # A Decision Node asks a question: This holds a reference to the question, and to the two child nodes.

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [56]:
def build_tree(rows, max_depth, curr_depth=0):
    """Builds the tree.
    
    Rules of recursion: 
        2) Start by checking for the base case (no further information gain). 
        3) Prepare for giant stack traces.
    """
    
    if (curr_depth<=max_depth):
        # Try partitioing the dataset on each of the unique attribute,
        # calculate the information gain,
        # and return the question that produces the highest gain.
        gain, question = find_the_best_split(rows)
    
        # Base case: no further info gain
        # Since we can ask no further questions,
        # we'll return a leaf.
        if gain == 0:
            return Leaf(rows)
    
        true_rows, false_rows = partitioner(rows, question)
        true_branch = build_tree(true_rows, max_depth, curr_depth + 1) # Recursively build the true branch.
        false_branch = build_tree(false_rows, max_depth, curr_depth + 1) # Recursively build the false branch.
    
        # Return a Decision_Node.
        # This records the best feature / value to ask at this point, as well as the branches to follow depending on the answer.
    
        return Decision_Node(question, true_branch, false_branch)
    
    return Leaf(rows)

In [57]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [58]:
def build_dt(X, y, attribute_types, options):
    rows = [sublist + [y[i % len(y)]] for i, sublist in enumerate(X)]
    return build_tree(rows, options["max_depth"])

In [59]:
class_counts(train)

{6: 168,
 7: 261,
 8: 437,
 9: 529,
 11: 371,
 13: 132,
 10: 482,
 12: 179,
 15: 51,
 14: 76,
 3: 7,
 4: 32,
 5: 68,
 16: 38,
 17: 30,
 19: 15,
 27: 2,
 18: 24,
 20: 10,
 21: 5,
 25: 1,
 23: 4,
 24: 2}

In [60]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.compare(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [61]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [62]:
X = abalone_df.drop(columns="Rings")
y = abalone_df["Rings"]
options = {"max_depth":1}
attribute_types = abalone_df.dtypes.apply(str).tolist()


my_tree = build_dt(X.values.tolist(), y.values.tolist(), attribute_types, options)
print_tree(my_tree)

Is Shell weight >= 0.145
--> True:
  Is Shell weight >= 0.2535
  --> True:
    Predict {20: 24, 16: 53, 19: 29, 12: 176, 10: 399, 11: 351, 15: 73, 18: 35, 13: 144, 14: 88, 9: 286, 21: 12, 17: 50, 7: 12, 8: 86, 22: 5, 26: 1, 23: 8, 29: 1, 6: 4, 27: 2, 25: 1, 24: 2}
  --> False:
    Predict {9: 294, 10: 172, 14: 35, 11: 104, 12: 71, 8: 280, 7: 73, 15: 26, 13: 47, 17: 8, 20: 2, 18: 6, 19: 3, 21: 2, 23: 1, 22: 1, 6: 14, 16: 10}
--> False:
  Is Diameter >= 0.225
  --> True:
    Predict {7: 289, 8: 196, 10: 63, 9: 108, 11: 32, 6: 194, 12: 20, 13: 12, 14: 3, 4: 4, 5: 37, 15: 3, 16: 4, 18: 1}
  --> False:
    Predict {5: 78, 4: 53, 7: 17, 9: 1, 1: 1, 3: 15, 6: 47, 8: 6, 2: 1}


In [63]:
for row in test:
    print ("Actual: %s. Predicted: %s" % (row[-1], print_leaf(classify(row, my_tree))))

Actual: 7. Predicted: {7: '29%', 8: '20%', 10: '6%', 9: '11%', 11: '3%', 6: '20%', 12: '2%', 13: '1%', 14: '0%', 4: '0%', 5: '3%', 15: '0%', 16: '0%', 18: '0%'}
Actual: 9. Predicted: {9: '25%', 10: '14%', 14: '3%', 11: '9%', 12: '6%', 8: '24%', 7: '6%', 15: '2%', 13: '4%', 17: '0%', 20: '0%', 18: '0%', 19: '0%', 21: '0%', 23: '0%', 22: '0%', 6: '1%', 16: '0%'}
Actual: 10. Predicted: {9: '25%', 10: '14%', 14: '3%', 11: '9%', 12: '6%', 8: '24%', 7: '6%', 15: '2%', 13: '4%', 17: '0%', 20: '0%', 18: '0%', 19: '0%', 21: '0%', 23: '0%', 22: '0%', 6: '1%', 16: '0%'}
Actual: 7. Predicted: {7: '29%', 8: '20%', 10: '6%', 9: '11%', 11: '3%', 6: '20%', 12: '2%', 13: '1%', 14: '0%', 4: '0%', 5: '3%', 15: '0%', 16: '0%', 18: '0%'}
Actual: 8. Predicted: {7: '29%', 8: '20%', 10: '6%', 9: '11%', 11: '3%', 6: '20%', 12: '2%', 13: '1%', 14: '0%', 4: '0%', 5: '3%', 15: '0%', 16: '0%', 18: '0%'}
Actual: 20. Predicted: {20: '1%', 16: '2%', 19: '1%', 12: '9%', 10: '21%', 11: '19%', 15: '3%', 18: '1%', 13: '7